Process:<br>
1-Load the data<br>
2-Preprocess the data(tokenizing punctualtion, lower case except for names, split)<br>
3-Create dictionary from the words<br>
4-Build and train the model<br>
5-Generate the new text<br>

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
import string
import warnings
import re
from tensorflow.contrib import legacy_seq2seq


In [2]:
#Load data
f = open("HarryPotterCh1_SorcererStone.txt","r") 
textbook = f.read()
#print(textbook)
f.close()

Here, we create our own function for preprocessing, tokenization as well as creating index of the words.

In [3]:
def preprocess_text(string):
    #Tokenize the punctuations in order to consider them as words
    string = string.replace("\n", " nextline ")
    string = string.replace(".", " periodmark")
    string = string.replace(":", " colonmark")
    string = string.replace(";", " semicolonmark")
    string = string.replace(",", " commamark")
    string = string.replace("?", " questionmark")
    string = string.replace("!", " exclamationmark")
    string = string.replace("...", " 3dots")
    string = string.replace("--", " 2dashes")
    string = string.replace("\"", " quotemark")
    string = string.replace("(", " leftparan")
    string = string.replace(")", " rightparan")
    
    #Names to remain capitalized
    Names = ['Harry', 'Potter', 'James','Jim', 'Abbott','George','Hannah', 'Susan', 'Bones', 'Minerva','McGonagall','Professor',
             'Sprout','Malfoy','Draco','Voldemort','Rubeus','Percy','Snape','Weasley','Hagrid','Fred','Scabbers','Hedwig',
            'Sirius','Hermione','Granger','Ronald','Peeves','Vernon','Dursley','Mrs.','Mr.','Norris','Argus','Filch','Nick','Charlie','Neville',
             'Quirrell','Dumbledore','Filch','Flitwick','McGonagall','McGuffin','Ollivander','Baron','Pomfrey','Gryffindor','Slytherin',
             'Ravenclaw','Hufflepuff']

    toLower = lambda x: " ".join( a if a in Names else a.lower()
            for a in x.split() )

    string= toLower(string)
    return string

In [4]:
text=preprocess_text(textbook)
#print(text)
text_words=text.split()
text_len=len(text_words)

In [5]:
#Create dictionary from list of words in text
def dictionary(words):
    #create list of words without their dupications 
    words=set(words)
    #map word to index
    indx = {key: i for i, key in enumerate(words)}
    return indx


In [6]:
#Convert from index to words
def get_by_key_dict(indx_word,words_dict):
    for word, indx in words_dict.iteritems():    
        if indx == indx_word:
            return word

In [7]:
words_index=dictionary(text_words)
words_index

{"periodmark'\xe2\x80\x9c": 0,
 'wrought-iron': 1,
 'both': 2739,
 'foul': 16,
 'four': 3,
 'woods': 4,
 'spiders': 5,
 'ornate': 25,
 'fiddling': 5681,
 'wizardry': 7,
 'Ronald': 8,
 "fluffy's": 9,
 'lord': 10,
 'flicking': 11,
 'three-thirty': 12,
 'sinking': 13,
 'figg': 14,
 "quotemarkwhat's": 15,
 'yellow': 2,
 'bringing': 113,
 'disturb': 18,
 'basics': 19,
 'wooden': 20,
 'wednesday': 21,
 'quotemarkhurry': 22,
 'specially': 23,
 'tired': 24,
 'hanging': 6,
 'bacon': 26,
 'second': 27,
 'crisply': 28,
 'sailed': 29,
 'scraped': 30,
 'leftparantwo': 31,
 'iron-gray': 32,
 'thunder': 33,
 'fingers': 34,
 "'smatter": 35,
 'pawed': 36,
 'quotemarkhold': 37,
 'pressed': 38,
 'Neville': 39,
 'hero': 40,
 '-then': 41,
 "norris's": 42,
 'here': 43,
 'reported': 44,
 'ashen-faced': 45,
 'shriek': 46,
 'kids': 47,
 'quotemarkhas': 48,
 'climbed': 49,
 'reports': 50,
 'quotemarkhad': 51,
 "i'd": 52,
 'transfixed': 53,
 "i'm": 54,
 'golden': 55,
 'explained': 56,
 'brought': 57,
 'stern': 5

Create sequences of 10 length (given 10 words as inputs, predict 1 word for output added to the previos words)

In [8]:
seq_len=10

In [9]:
def  create_model_inputs(batch_size):
    '''Define model inputs'''
    
    #Model's placeholders for inputs
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs,targets,keep_prob

In [10]:
def  build_RNN(vocabulary_size,embedding_size,inputs,seq_len,num_hidden,lstm_layer_numbers,keep_prob,batch_size):
    '''Build RNN'''
    #Embedding Layer
    '''Intialize embeddings for the words. Embedding layer connects the words to the LSTM layers (words are embedded to the embedding_size vectors instead of vocabulary size vectors or one hot vectors). Here, provided by tensorflow, we used random_uniform distribution to create embeddings'''
    #tf.AUTO_REUSE for reuisng the same scope for generating as for traning
    with tf.variable_scope('rnn1', reuse=tf.AUTO_REUSE):
        embedding = tf.Variable(tf.random_uniform((vocabulary_size, embedding_size), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, inputs)
        #Define LSTM layers
        lstms=[]
        for i in range(lstm_layer_numbers):
            lstms.append(tf.contrib.rnn.BasicLSTMCell(num_hidden))
        # Add regularization dropout to the LSTM cells
        drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) for lstm in lstms]
        # Stack up multiple LSTM layers
        stacked_lstm = tf.contrib.rnn.MultiRNNCell(drops)
        # Getting the initial state
        initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        #outputs, final_state = tf.nn.dynamic_rnn(stacked_lstm, embed, initial_state=initial_state)
        #need to unstack the sequence of input into a list of tensors
        seq_input = [tf.squeeze(i,[1]) for i in tf.split(embed,seq_len,1)] 

        outputs, final_state = legacy_seq2seq.rnn_decoder(seq_input, initial_state, stacked_lstm, loop_function=None)

    return initial_state, outputs, final_state

In [11]:
def get_batches(text_words,text_len,seq_len, batch_size,number_of_words_in_one_batch,n_batches):
    '''Using generator to return batches'''
    
    #This makes the input data to be compatible with seq_len
    text_all_batches = text_words[:n_batches*number_of_words_in_one_batch]
    index_text_all_batches=[]
    for i in text_all_batches:
        if i in words_index:
            index_text_all_batches.append(words_index[i])
        
    #index_text_all_batches={v for k,v in words_index.items() if k in text_all_batches}
    #get word index for words for batches
    input_seq=list(index_text_all_batches)
    output_seq=input_seq
    output_seq.append(output_seq.pop(output_seq[0]))
    for ii in range(0, len(text_all_batches), number_of_words_in_one_batch):
        yield input_seq[ii:ii+number_of_words_in_one_batch], output_seq[ii:ii+number_of_words_in_one_batch]

In [12]:
#Define Parameters
# number of units
n_input= len(words_index)
num_hidden = 512
lstm_layer_numbers=3
embed_size=512
batch_size= 512
learning_rate=0.001

Create a graph for training

In [13]:
graph0 = tf.Graph()
#There exits a global default graph created by tenserflow, for new graphs we need to set them as a default graph
with graph0.as_default():
    inputs,targets,keep_prob=create_model_inputs(batch_size)
    initial_state, outputs, final_state = build_RNN(n_input,embed_size,inputs,seq_len,num_hidden,lstm_layer_numbers,keep_prob,batch_size)
    # Loss and optimizer
    logits = tf.contrib.layers.fully_connected(outputs, n_input, activation_fn=None)
    
    probs = tf.nn.softmax(logits, name='probs')
    print(probs.shape)
    cost =  tf.contrib.seq2seq.sequence_loss(
            logits,
            targets,
            tf.ones([batch_size, (seq_len)])    
        )                                   

    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    # Using gradian clipping for exploding gradients
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients) 
    
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()

#Execute the graph for training
with tf.Session(graph=graph0) as sess:
    sess = tf.Session(graph=graph0)
    sess.run(init_op)
    number_of_words_in_one_batch= seq_len*batch_size
    n_batches = text_len//number_of_words_in_one_batch
    epochs = 501
    for epoch in range(epochs):
        state = sess.run(initial_state)
        avg_cost_train = 0 
        avg_acc_train= 0
        for ii, (x, y) in enumerate(get_batches(text_words,text_len,seq_len,batch_size,number_of_words_in_one_batch,n_batches), 1):
            #need to reshape y to feed it to targets
            y = np.array(y).reshape(batch_size,(seq_len))
            x = np.array(x).reshape(batch_size,(seq_len))

            state, loss, _= sess.run([final_state, cost,train_op], feed_dict={inputs: x,
                                                            targets: y,keep_prob: 0.8,initial_state: state})

            avg_cost_train += loss / n_batches
        if(epoch%10==0):
            print(epoch)
            print("cost_train=", avg_cost_train) 
    #Save the model into a file 
    checkpoint="./model/savedmodel.ckpt"
    save_path = saver.save(sess, checkpoint)


(10, 512, 6691)
0
('cost_train=', 6.972886662734183)
10
('cost_train=', 6.184860681232653)
20
('cost_train=', 6.1766271340219605)
30
('cost_train=', 6.1627941382558715)
40
('cost_train=', 6.1588198511224039)
50
('cost_train=', 6.1411596850344994)
60
('cost_train=', 6.0770749794809431)
70
('cost_train=', 5.9195501678868334)
80
('cost_train=', 5.6222850899947314)
90
('cost_train=', 5.2673218626725049)
100
('cost_train=', 4.9383979094655892)
110
('cost_train=', 4.8178091049194336)
120
('cost_train=', 4.4054311953092871)
130
('cost_train=', 4.0410112958205371)
140
('cost_train=', 3.7694851724725025)
150
('cost_train=', 3.4515780900654045)
160
('cost_train=', 3.0658338195399231)
170
('cost_train=', 2.8562086632377226)
180
('cost_train=', 2.5433502071782166)
190
('cost_train=', 2.3181784780401933)
200
('cost_train=', 2.1251794538999857)
210
('cost_train=', 1.7791131923073218)
220
('cost_train=', 1.5872602839218943)
230
('cost_train=', 1.4524070996987191)
240
('cost_train=', 1.281992937389173

In [14]:
batch_size=1
#use the same sequence length as for trained model to generate the new words
seq_len=10

Craete a graph for generating text<br>
Based on the train model, each time it uses 10 previous words to generate the next word (therfore, first define 10 prime words to begin generating 11th word and then consider 2th to 11th words for generating the 12th word and so on)

In [15]:
tf.reset_default_graph()
graph1 = tf.Graph()

with graph1.as_default():
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    inputs = tf.placeholder(tf.int32, [batch_size, None], name='inputs')
    initial_state, outputs, final_state = build_RNN(n_input,embed_size,inputs,seq_len,num_hidden,lstm_layer_numbers,keep_prob,batch_size)
    logits = tf.contrib.layers.fully_connected(outputs, n_input, activation_fn=None)
    probs = tf.nn.softmax(logits, name='probs')
    init_op1 = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
#Execute the graph1 for generating text
with tf.Session(graph=graph1) as sess2:
    #This part to compare varibles in checkpoints with what we have
    var_name_list = [v.name for v in tf.trainable_variables()]
    #print(var_name_list)
    from tensorflow.python import pywrap_tensorflow

    reader = pywrap_tensorflow.NewCheckpointReader(checkpoint)
    var_to_shape_map = reader.get_variable_to_shape_map()
    #print(var_to_shape_map)
    
    #Execute the graph to generate the text
    sess2.run(init_op1)
    
    #Number of words to generate 
    num_gen=500
    
    # Load the model
    saved = tf.train.import_meta_graph('./model/savedmodel.ckpt.meta')
    saved.restore(sess2, checkpoint)
    
    #Just for the record 
    saved_dict = {}
    for x in tf.trainable_variables():
          saved_dict[x.name] = x
    
    #10 first words to begin with
    start_word="Harry Potter went to see the street even it was"
    start_words=start_word.split(" ")
    print("Prime words: ",start_words)
    
    #The sentence of text we have so far as a list of words' indexes
    genertaed_sentence=[words_index[w] for w in start_words]
    genertaed_sentence=[genertaed_sentence]
    
    state = sess2.run(initial_state)
    #Choose the last 10 words we have from text to predict the next word
    last_words=[(genertaed_sentence[0])[-10:]]
    
    
    for i in range(0,num_gen):
        #seq_len is 10
        seq_len=len(last_words[0])
        
        #print("last_words",last_words)
        
        next_word = np.zeros((1,seq_len))
        next_word = [w for w in last_words]
        #print(np.array(next_word).shape,inputs.shape,type(next_word),next_word)
        next_word = np.asarray(next_word) 

        #next_word = next_word.reshape(batch_size,seq_len)
            
        #print("next_word",next_word)
         #next_word = np.array(next_word).reshape(batch_size,(seq_len))
        prediction,state= sess2.run([probs,final_state], feed_dict={inputs: next_word,
                                                                    keep_prob: 0.8,initial_state: state})
        #print("Prediction's shape",prediction.shape," Prediction:",prediction)
        #print("Element we choose for prediction: ",prediction[len(last_words[0])-1,0])
        #Based on prediction's shape still not sure about part len(last_words[0])-1, which element to choose
        
        #Next predicted word by choosing the word with max probability
        next_predicted_word = np.argmax(prediction[len(last_words[0])-1,0])
        
        #append the new word to the previous sentences
        genertaed_sentence[0].append(next_predicted_word)
        #save in last_word to use it in for loop
        last_words=[(genertaed_sentence[0])[-10:]]

#Conver index to words
list_gen=[get_by_key_dict(word_int,words_index) for word_int in genertaed_sentence[0]]
sen=' '.join(list_gen)
#Convert back the tokens for punctuations
sen=sen.replace("nextline", "\n")
sen=sen.replace("periodmark", ".")
sen=sen.replace("colonmark", ":")
sen=sen.replace("commamark", ",")
sen=sen.replace("semicommamark", ";")
sen=sen.replace("questionmark", "?")
sen=sen.replace("exclamationmark", "!")
sen=sen.replace("3dots", "...")
sen=sen.replace("2dashes", "--")
sen=sen.replace("quotemark", "\"")
sen=sen.replace("leftparan", "(")
sen=sen.replace("rightparan", ")")
#Print the whole text
print(sen)

INFO:tensorflow:Restoring parameters from ./model/savedmodel.ckpt
('Prime words: ', ['Harry', 'Potter', 'went', 'to', 'see', 'the', 'street', 'even', 'it', 'was'])
Harry Potter went to see the street even it was abou' . 
 in almost 
 game on . talking the important they their he me . about go semi: -- go get table flamel very ? i of called on and moment along which " don't suddenly what his 
 second McGonagall the 
 "i couldn't ! " common he's before abou' . manage , material to cross once ron 
 " looked "no his silver a . a one set badge -- that into would ! the . coins quietly no his 
 other their McGonagall his common the unicorn Gryffindor Snape did mirror dying i was hung where Harry you as their the were " a 
 last -- it -- "and . them how 
 were as say dead " " who as . ? around Harry a you . same ? . the 
 any shoulder the new had Percy flames him . it Harry it go was or -- his 
 the come . . . "but Harry McGonagall 
 Hagrid finished said away hardly " Weasley 
 , . . every top